In [26]:
import pandas as pd
import numpy as np
# Load data
train = pd.read_csv(r'C:\Users\teble\rossmann-sales-prediction\rossmann-sales-prediction\data\train.csv')
test = pd.read_csv(r'C:\Users\teble\rossmann-sales-prediction\rossmann-sales-prediction\data\test.csv', low_memory=False)
store = pd.read_csv(r'C:\Users\teble\rossmann-sales-prediction\rossmann-sales-prediction\data\store.csv')
sample_submission = pd.read_csv(r'C:\Users\teble\rossmann-sales-prediction\rossmann-sales-prediction\data\sample_submission.csv')
train['Date'] = pd.to_datetime(train['Date'])
train['StateHoliday'] = train['StateHoliday'].astype('category')
print(train.isnull().sum())


# Print the data types of the columns in the train dataset
print(train.dtypes)

C:\Users\teble\AppData\Local\Temp\ipykernel_26348\3392884638.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(r'C:\Users\teble\rossmann-sales-prediction\rossmann-sales-prediction\data\train.csv')


Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64
Store                     int64
DayOfWeek                 int64
Date             datetime64[ns]
Sales                     int64
Customers                 int64
Open                      int64
Promo                     int64
StateHoliday           category
SchoolHoliday             int64
dtype: object


In [27]:
# Merge data on 'Store' column
train = train.merge(store, how='left', on='Store')
test = test.merge(store, how='left', on='Store')

In [28]:
# Initial exploration
print(train.info())  # Shows basic info about the dataset
print(train.describe())  # Descriptive statistics
print(train.head())  # First few rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Store                      1017209 non-null  int64         
 1   DayOfWeek                  1017209 non-null  int64         
 2   Date                       1017209 non-null  datetime64[ns]
 3   Sales                      1017209 non-null  int64         
 4   Customers                  1017209 non-null  int64         
 5   Open                       1017209 non-null  int64         
 6   Promo                      1017209 non-null  int64         
 7   StateHoliday               1017209 non-null  category      
 8   SchoolHoliday              1017209 non-null  int64         
 9   StoreType                  1017209 non-null  object        
 10  Assortment                 1017209 non-null  object        
 11  CompetitionDistance        1014567 no

In [15]:
# Check for missing values
print(train.isnull().sum())

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance            2642
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
dtype: int64


In [34]:
# Fill missing values for numerical columns with the median and reassign the column
train['CompetitionDistance'] = train['CompetitionDistance'].fillna(train['CompetitionDistance'].median())

# Similarly, you can handle other columns with missing values
train['Sales'] = train['Sales'].fillna(train['Sales'].median())
train['Open'] = train['Open'].fillna(train['Open'].mode()[0])  # For categorical columns, you can fill with mode

# Check again after filling
print(train.isnull().sum())


Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance               0
CompetitionOpenSinceMonth    258728
CompetitionOpenSinceYear     258728
Promo2                            0
Promo2SinceWeek              399497
Promo2SinceYear              399497
PromoInterval                399497
dtype: int64


In [35]:
# Remove rows with zero sales (outliers)
train = train[train['Sales'] > 0]

# Identifying and handling outliers (Z-score or IQR method)
from scipy import stats

In [31]:
# Calculate Z-scores for the 'Sales' column
z_scores = stats.zscore(train['Sales'])
train = train[(z_scores < 3)]  # Remove extreme outliers with Z-score > 3

# Alternatively, using the IQR method for 'Sales'
Q1 = train['Sales'].quantile(0.25)
Q3 = train['Sales'].quantile(0.75)
IQR = Q3 - Q1
train = train[(train['Sales'] >= (Q1 - 1.5 * IQR)) & (train['Sales'] <= (Q3 + 1.5 * IQR))]

# Check the data after cleaning
print(train.head())

   Store  DayOfWeek       Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5 2015-07-31   5263        555     1      1            0   
1      2          5 2015-07-31   6064        625     1      1            0   
2      3          5 2015-07-31   8314        821     1      1            0   
4      5          5 2015-07-31   4822        559     1      1            0   
5      6          5 2015-07-31   5651        589     1      1            0   

   SchoolHoliday StoreType Assortment  CompetitionDistance  \
0              1         c          a               1270.0   
1              1         a          a                570.0   
2              1         a          a              14130.0   
4              1         a          a              29910.0   
5              1         a          a                310.0   

   CompetitionOpenSinceMonth  CompetitionOpenSinceYear  Promo2  \
0                        9.0                    2008.0       0   
1                       11